Cell 1 — Mount Drive

In [1]:
# 1) Mount Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Cell 2 — Set inputs

In [2]:
# 2) Project and input file
PROJECT_DIR = "/content/drive/MyDrive/WorstellWitnessWaybackMachine"  # <-- change if your path differs
DOCX_NAME   = "1600-1610.docx"                                       # <-- your Word file in /manuscript/
TITLE       = "1600–1610 — First Glimpses"
SUMMARY     = "One ship among thousands; fragile English outpost; Indigenous majority everywhere."

# Derived paths
import os, re, pathlib
DOCX_PATH = f"{PROJECT_DIR}/manuscript/{DOCX_NAME}"
MD_SLUG   = re.sub(r"[–—]", "-", DOCX_NAME).replace(".docx",".md")   # normalize en-dash/em-dash
OUT_MD    = f"{PROJECT_DIR}/docs/decades/{MD_SLUG}"
DL_DST    = f"{PROJECT_DIR}/docs/downloads/{DOCX_NAME}"

print("DOCX →", DOCX_PATH)
print("MD   →", OUT_MD)
print("DL   →", DL_DST)

DOCX → /content/drive/MyDrive/WorstellWitnessWaybackMachine/manuscript/1600-1610.docx
MD   → /content/drive/MyDrive/WorstellWitnessWaybackMachine/docs/decades/1600-1610.md
DL   → /content/drive/MyDrive/WorstellWitnessWaybackMachine/docs/downloads/1600-1610.docx


Cell 3 — Ensure folders and (if needed) upload the DOCX

In [3]:
# 3) Ensure folders exist (safe to re-run)
import pathlib
for p in ["manuscript","docs/decades","docs/downloads"]:
    pathlib.Path(PROJECT_DIR, p).mkdir(parents=True, exist_ok=True)

# Optional helper: prompt to upload the DOCX into /manuscript if it's not there yet
from pathlib import Path
if not Path(DOCX_PATH).exists():
    print("⚠️ DOCX not found:", DOCX_PATH)
    print("→ Drag your DOCX into this Drive folder, then re-run this cell:")
    print(f"  {PROJECT_DIR}/manuscript")
else:
    print("✅ Found:", DOCX_PATH)

✅ Found: /content/drive/MyDrive/WorstellWitnessWaybackMachine/manuscript/1600-1610.docx


Cell 4 — Convert DOCX→Markdown + add front-matter + copy DOCX to downloads

In [4]:
# 4) Install Pandoc and convert DOCX → GitHub-flavored Markdown
!apt-get -y -qq install pandoc
!pandoc "$DOCX_PATH" -f docx -t gfm -o "$OUT_MD"

# Prepend YAML front matter and a 'Crowd size' callout if not present
with open(OUT_MD, "r+", encoding="utf-8") as f:
    body = f.read()
    if not body.lstrip().startswith("---"):
        header = (
            f"---\n"
            f"title: {TITLE}\n"
            f"summary: {SUMMARY}\n"
            f"---\n\n"
            f"!!! info \"Crowd size (feel, not a census)\"\n"
            f"    - Add your ranges here.\n\n"
        )
        f.seek(0); f.write(header + body); f.truncate()

# Copy DOCX into docs/downloads for readers
import shutil
shutil.copyfile(DOCX_PATH, DL_DST)
print("✅ Converted and copied download →", DL_DST)

Selecting previously unselected package libcmark-gfm0.29.0.gfm.3:amd64.
(Reading database ... 126435 files and directories currently installed.)
Preparing to unpack .../libcmark-gfm0.29.0.gfm.3_0.29.0.gfm.3-3_amd64.deb ...
Unpacking libcmark-gfm0.29.0.gfm.3:amd64 (0.29.0.gfm.3-3) ...
Selecting previously unselected package libcmark-gfm-extensions0.29.0.gfm.3:amd64.
Preparing to unpack .../libcmark-gfm-extensions0.29.0.gfm.3_0.29.0.gfm.3-3_amd64.deb ...
Unpacking libcmark-gfm-extensions0.29.0.gfm.3:amd64 (0.29.0.gfm.3-3) ...
Selecting previously unselected package pandoc-data.
Preparing to unpack .../pandoc-data_2.9.2.1-3ubuntu2_all.deb ...
Unpacking pandoc-data (2.9.2.1-3ubuntu2) ...
Selecting previously unselected package pandoc.
Preparing to unpack .../pandoc_2.9.2.1-3ubuntu2_amd64.deb ...
Unpacking pandoc (2.9.2.1-3ubuntu2) ...
Setting up libcmark-gfm0.29.0.gfm.3:amd64 (0.29.0.gfm.3-3) ...
Setting up libcmark-gfm-extensions0.29.0.gfm.3:amd64 (0.29.0.gfm.3-3) ...
Setting up pandoc-da

Cell 5 — Add page to mkdocs.yml navigation (if missing)

In [5]:
# 5) Append decade to navigation (mkdocs.yml) if not present
import re, pathlib

cfg_path = pathlib.Path(PROJECT_DIR, "mkdocs.yml")
cfg = cfg_path.read_text(encoding="utf-8")

nav_line = f"      - {TITLE.split(' — ')[0]}: decades/{MD_SLUG}\n"  # shows just the decade label on nav

if f"decades/{MD_SLUG}" not in cfg:
    if "Decades:\n" in cfg:
        cfg = re.sub(r"(Decades:\n)", r"\1" + nav_line, cfg, count=1)
    else:
        # Create a basic Decades section near the top
        cfg = re.sub(r"(nav:\n)", r"\1  - Decades:\n" + nav_line, cfg, count=1)
    cfg_path.write_text(cfg, encoding="utf-8")
    print("✅ Updated mkdocs.yml nav.")
else:
    print("ℹ️ Nav already contains this page.")

ℹ️ Nav already contains this page.


Cell 6 — Build & preview locally

In [6]:
# 6) Build a local preview
!python -m pip install -q mkdocs mkdocs-material
%cd $PROJECT_DIR
!mkdocs build -q

# Show the built homepage in output (you can click through to the new decade page)
from IPython.display import HTML
HTML(open("site/index.html","r",encoding="utf-8").read())

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 107.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.9/397.9 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.2/266.2 kB 19.8 MB/s eta 0:00:00
/content/drive/MyDrive/WorstellWitnessWaybackMachine
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:44: SyntaxWarning: invalid escape sequence '\.'
  re_han_default = re.compile("([\u4E00-\u9FD5a-zA-Z0-9+#&\._%\-]+)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/__init__.py:46: SyntaxWarning: invalid escape sequence '\s'
  re_skip_default = re.compile("(\r\n|\s)", re.U)
/usr/local/lib/python3.12/dist-packages/jieba/finalseg/__init__.py:78: SyntaxWarning: invalid escape sequence '\.'
  re_skip = re.compile("([a-zA-Z0-9]+(?:\.\d+)?%?)")


Optional) Cell 7 — Add a download link to the top of the new page

In [7]:
# 7) Insert a "Download the Word version" link if it's not there already
inject_line = f"\n[Download the Word version](../downloads/{DOCX_NAME})\n"
with open(OUT_MD, "r+", encoding="utf-8") as f:
    body = f.read()
    if f"(../downloads/{DOCX_NAME})" not in body:
        f.seek(0); f.write(body + inject_line); f.truncate()
        print("✅ Added download link to the page.")
    else:
        print("ℹ️ Download link already present.")

# Rebuild to reflect the link
%cd $PROJECT_DIR
!mkdocs build -q

✅ Added download link to the page.
/content/drive/MyDrive/WorstellWitnessWaybackMachine


Optional push (use your 01 notebook or this snippet)

If you want to push right from this notebook and your remote was set to a tokenized URL:

In [8]:
# Optional push: works if your 'origin' remote includes your token
%cd $PROJECT_DIR
!git add -A
!git commit -m "Add decade: {DOCX_NAME}" || true
!git pull --rebase origin main || true
!git push -u origin main

/content/drive/MyDrive/WorstellWitnessWaybackMachine
[main d02de76] Add decade: 1600-1610.docx
 1 file changed, 159 insertions(+), 25 deletions(-)
 rewrite docs/decades/1600-1610.md (88%)
From https://github.com/BillWorstell/WorstellWitnessWaybackMachine
 * branch            main       -> FETCH_HEAD
Current branch main is up to date.
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 2 threads
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 3.80 KiB | 155.00 KiB/s, done.
Total 5 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/BillWorstell/WorstellWitnessWaybackMachine.git
   489d9e9..d02de76  main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.
